In [4]:
import os
from PIL import Image
import numpy as np
n=4

# Function to perform logistic map iteration
def logistic_map(x, a, n):
    for _ in range(n):
        x = a * x * (1 - x)
    return x

# Function to perform Zigzag pattern and rotation
def zigzag_rotation(block):
    zigzag = np.ravel(block, order='F')
    rotated_block = np.rot90(zigzag.reshape(block.shape), k=1)
    return rotated_block

In [16]:
initial_condition = np.random.rand()
h = 2 ** n
M, N = plain_image.shape
(logistic_map(initial_condition, a, n0 + M * N))


[0.4172870261105104,
 0.924002543011355,
 0.26684300537563543,
 0.7434237002593893,
 0.7248306281777026,
 0.7579145168062346,
 0.6972243876783049,
 0.8021896554386873,
 0.6029893661542204,
 0.9096941237447773,
 0.31217275488219653,
 0.8159395187675349,
 0.5706924378275661,
 0.9310098010891836,
 0.24407609518723697,
 0.7011112287924737,
 0.7963062398836089,
 0.6163699263829413,
 0.8985405528877894,
 0.3464286252748014,
 0.8603801648870479,
 0.4564793196727397,
 0.9428026114586391,
 0.20491821968455615,
 0.6191216231182989,
 0.8960781478415154,
 0.3538639830492517,
 0.8688482052891283,
 0.43301380552895835,
 0.9329488290510937,
 0.23771018340849012,
 0.6885753980270899,
 0.8148694131839135,
 0.5732575600366283,
 0.9296066536104232,
 0.24866486805988475,
 0.7099564755200504,
 0.7824894578713543,
 0.6467588835278837,
 0.8681549544014664,
 0.43495533228868594,
 0.933922926568834,
 0.23450139643216544,
 0.6821398677134134,
 0.823935260639493,
 0.5512505982748103,
 0.9400188294706012,
 0.2142

In [32]:
import os
from PIL import Image
import numpy as np
n=4

# Function to perform logistic map iteration
def logistic_map(x, a, n):
 
    sequence = [x]
    for _ in range(n - 1):
        x = a * x * (1 - x)
        sequence.append(x)
    return sequence


# Function to perform Zigzag pattern and rotation
def zigzag_rotation(block):
    zigzag = np.ravel(block, order='F')
    rotated_block = np.rot90(zigzag.reshape(block.shape), k=1)
    return rotated_block

# Function to encrypt the image
def conceal_encrypt(plain_image, a, n0):
    M, N = plain_image.shape
    h = 2 ** n
    block_size = h ** 2
    num_blocks = M * N // block_size

    # Step 2: Generate random numbers for each block
    random_numbers = np.random.choice(range(2, n + 1), size=num_blocks)

    # Step 6: Perform Zigzag pattern and rotation
    encrypted_blocks = []

    for i in range(num_blocks):
        # Step 4: Divide a block into sub-blocks or keep without dividing based on Ri
        block = plain_image[i * h:i * h + h, :]  # Assign block even if not divided
        incomplete_rows = h - block.shape[0]
        if incomplete_rows > 0:
            block = np.pad(block, ((incomplete_rows, 0), (0, 0)), mode='constant')  # Pad with zeros
        #print(f"Block {i} shape before transformation: {block.shape}")
        if block.shape[0] > 0:
            encrypted_blocks.append(zigzag_rotation(block))
            print(f"Block {i} shape after filtering: {block.shape}")
        print(f"Encrypted block shapes: {[block.shape for block in encrypted_blocks]}")
        if random_numbers[i] > n / 2:
            sub_blocks = np.array_split(plain_image[i * h:i * h + h, :], h, axis=1)
            transformed_sub_blocks = [zigzag_rotation(sub_block) for sub_block in sub_blocks]
            encrypted_blocks.extend(transformed_sub_blocks)
        else:
            block = plain_image[i * h:i * h + h, :]
            if block.shape[0] < h:  # Check if block is incomplete (last block)
                # Pad the block with zeros (or another strategy) to match expected size
                block = np.pad(block, ((h - block.shape[0], 0), (0, 0)), mode='constant')
            encrypted_blocks.append(zigzag_rotation(block))

    # Step 7: Generate random vector r
    r = np.random.randint(0, 256, size=num_blocks)

    # Step 8: Random permutation of image blocks based on r
    permuted_blocks = [encrypted_blocks[i] for i in np.argsort(r)]

    # Step 9-13: Generate the chaotic sequence S and create the keystream K
    initial_condition = np.random.rand()
    chaotic_sequence = [logistic_map(initial_condition, a, n0 + M * N)[0] for _ in range(num_blocks)]


    keystream = [int(np.floor(s * 10**14) % 256) for s in chaotic_sequence]

    # Step 14-17: Encrypt the image
    flat_image = np.concatenate([block for block in encrypted_blocks]).ravel(order='F')

    encrypted_image = np.bitwise_xor(flat_image, keystream)
    encrypted_matrix = np.reshape(encrypted_image, plain_image.shape, order='F')

    return encrypted_matrix

# Function to read image from a folder
def read_image_from_folder(folder_path, image_name):
    image_path = os.path.join(folder_path, image_name)
    return np.array(Image.open(image_path).convert('L'))  # Convert to grayscale

# Function to save image to a folder
def save_image_to_folder(folder_path, image_name, image_data):
    image_path = os.path.join(folder_path, image_name)
    Image.fromarray(image_data).convert('L').save(image_path)  # Save as grayscale

# Set your folder path and image file name
folder_path = os.path.join("")
image_name = "normal (1).jpg"

# Read the image
plain_image = read_image_from_folder(folder_path, image_name)

# Set your parameters
a = 3.8
n0 = 100

# Encrypt the image
encrypted_image = conceal_encrypt(plain_image, a, n0)

# Save the encrypted image
encrypted_image_name = "encrypted_image.jpg"
save_image_to_folder(folder_path, encrypted_image_name, encrypted_image)


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



Block 769 shape after filtering: (16, 412)
Encrypted block shapes: [(412, 16), (412, 16), (412, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (25, 16), (25, 16), (25, 16), (25, 16), (412, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (25, 16), (25, 16), (25, 16), (25, 16), (412, 16), (412, 16), (412, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (25, 16), (25, 16), (25, 16), (25, 16), (412, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (25, 16), (25, 16), (25, 16), (25, 16), (412, 16), (412, 16), (412, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (26, 16), (25, 16), (25, 16), (25, 16), (25, 16), (412, 16), 

ValueError: all the input array dimensions for the concatenation axis must match exactly, but along dimension 1, the array at index 0 has size 16 and the array at index 316 has size 5

In [23]:
M, N = plain_image.shape
h = 2 ** n
block_size = h ** 2
num_blocks = M * N // block_size
for i in range(num_blocks):
    ...
    block = plain_image[i * h:i * h + h, :]
    print(f"Original block shape: {block.shape}")
    transformed_block = zigzag_rotation(block)
    print(f"Transformed block shape: {transformed_block.shape}")


Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412, 16)
Original block shape: (16, 412)
Transformed block shape: (412,

In [20]:
encrypted_blocks.extend(transformed_sub_blocks)
print(f"Added sub-block shapes: {[block.shape for block in transformed_sub_blocks]}")


NameError: name 'encrypted_blocks' is not defined